# Test handling of OAuth 2.0 tokens with Fitbit Web API

## Import existing tokens from JSON file

In [7]:
import json
import base64

with open('tokens/tokens.json') as f:
    token_dict = json.load(f)

client_id: str = token_dict['client_id']
client_secret: str = token_dict['client_secret']
access_token: str = token_dict['access_token']
refresh_token: str = token_dict['refresh_token']
# basic_token is necessary for the authorization for token refresh request
basic_token: str = base64.b64encode(f"{client_id}:{client_secret}".encode()).decode()
# print(basic_token)

## Use access token to request data from `heartrate` endpoint

In [10]:
import requests

date: str = "today"
headers = {
    "Authorization": f"Bearer {access_token}",
}
response = requests.get(f"https://api.fitbit.com/1/user/-/activities/heart/date/{date}/1d.json", headers=headers)
print(response.status_code)
hr_dict = json.loads(response.content)
print(hr_dict)
activity_minutes = hr_dict \
    .get('activities-heart')[0] \
    .get('value') \
    .get('customHeartRateZones')[1] \
    .get('minutes')

print(activity_minutes)

200
{'activities-heart': [{'dateTime': '2023-01-29', 'value': {'customHeartRateZones': [{'caloriesOut': 11.7213, 'max': 90, 'min': 30, 'minutes': 6, 'name': 'Below'}, {'caloriesOut': 1.4487, 'max': 200, 'min': 90, 'minutes': 1, 'name': 'Custom Zone'}, {'caloriesOut': 0, 'max': 220, 'min': 200, 'minutes': 0, 'name': 'Above'}], 'heartRateZones': [{'caloriesOut': 775.8447, 'max': 112, 'min': 30, 'minutes': 1440, 'name': 'Out of Range'}, {'caloriesOut': 0, 'max': 137, 'min': 112, 'minutes': 0, 'name': 'Fat Burn'}, {'caloriesOut': 0, 'max': 168, 'min': 137, 'minutes': 0, 'name': 'Cardio'}, {'caloriesOut': 0, 'max': 220, 'min': 168, 'minutes': 0, 'name': 'Peak'}]}}], 'activities-heart-intraday': {'dataset': [{'time': '00:00:00', 'value': 73}, {'time': '00:01:00', 'value': 71}, {'time': '00:02:00', 'value': 84}, {'time': '00:03:00', 'value': 92}, {'time': '09:44:00', 'value': 76}, {'time': '09:45:00', 'value': 74}, {'time': '09:46:00', 'value': 74}], 'datasetInterval': 1, 'datasetType': 'minu

## Refresh token pair `(access_token, refresh_token)` using current `refresh_token`

In [3]:
header = {
    "Authorization": f"Basic {basic_token}",
    "Content-Type": "application/x-www-form-urlencoded",
}

data = {
    "grant_type": "refresh_token",
    "client_id": client_id,
    "refresh_token": refresh_token,
}

response_dict = json.loads(requests.post("https://api.fitbit.com/oauth2/token", headers=header, data=data).content)
access_token = response_dict['access_token']
refresh_token = response_dict['refresh_token']
# print(refresh_token)

token_dict.update({'refresh_token': refresh_token, 'access_token': access_token})

with open("tokens/tokens.json", "w") as f:
    json.dump(token_dict, f, ensure_ascii=False, indent=4)